In [28]:
# Import the dependencies.
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests, time
from config import weather_api_key
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []

# Starting URL for Weather Map API Call.
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

597

In [73]:
# Initiate a dictionary of cities not found and error code
cities_not_located = {}

# Initiate list for city data
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval    ")
print("----------------------------")


# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    #Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >=50):
        set_count += 1
        record_count = 1
        print(f"Processing Set {set_count}")

#     if set_count == 2:
#         break
        
        
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city
    
    # log the URL, record, and set numbers and the city
    # print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_desc = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]

        # Conditionals for rain
        try:
            city_rainfall = city_weather["rain"]["1h"]
        except:
            city_rainfall = 0
            pass

        # Conditional for snow
        try:
            city_snowfall = city_weather["snow"]["1h"]
        except:
            city_snowfall = 0
            pass

        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,          
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Weather Description": city_weather_desc,
                          "Rainfall": city_rainfall,
                          "Snowfall": city_snowfall,                          
                          "Country": city_country,
                          "Date": city_date})
    #If an error is experienced, skip the city.
    except:
        err = sys.exc_info()[0]
        cities_not_located[i]=({"City":city,"Error Code":err})
#         print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval    
----------------------------
Processing Set 2
Processing Set 3
Processing Set 4
Processing Set 5
Processing Set 6
Processing Set 7
Processing Set 8
Processing Set 9
Processing Set 10
Processing Set 11
Processing Set 12
-----------------------------
Data Retrieval Complete      
-----------------------------


In [75]:
print(len(city_data))
print(len(cities_not_located))

538
59


In [77]:
# Export Path
export_path=os.path.join("Analysis","WeatherPy_challenge.csv")

pd.DataFrame(city_data).to_csv(export_path)

In [83]:
city_weather_df = pd.DataFrame(city_data)

citiesW_rain = city_weather_df[city_weather_df['Rainfall'] > 0]["City"].count()
print(citiesW_rain)

citiesW_snow = city_weather_df[city_weather_df['Snowfall'] > 0]["City"].count()
print(citiesW_snow)

45
6
